In [ ]:
import json
import pickle
import numpy as np
import matplotlib.pyplot as plt
from brian2 import *

%matplotlib inline

In [ ]:
# cell_type = 'regular_spiking'
# cell_type = 'adapting_spiking'
# cell_type = 'initial_bursting'
# cell_type = 'regular_bursting'
# cell_type = 'delayed_regular_bursting'
# cell_type = 'fast_spiking'
# cell_type = 'thorny'
cell_type = 'a-thorny'
config_file = f'../DL/configs/{cell_type}.json'
params = json.load(open(config_file))

In [ ]:
Cm      = params['C_m'] * pF
gL      = params['g_L'] * nS
EL      = params['E_L'] * mV
VT      = params['V_th'] * mV
Vpeak   = params['V_peak'] * mV
Vreset  = params['V_reset'] * mV
DeltaT  = params['Delta_T'] * mV
tauw    = params['tau_w'] * ms
a       = params['a'] * nS
b       = params['b'] * pA
tarp    = params['tau_arp'] * ms if 'tau_arp' in params else 0 * ms

In [ ]:
eqs = """
dV/dt = (-gL * (V - EL) + gL * DeltaT * 
         exp((V - VT) / DeltaT) + I - w) / Cm : volt (unless refractory)
dw/dt = (a * (V - EL) - w) / tauw : amp
"""

cell = NeuronGroup(1, model=eqs,
                   threshold='V>Vpeak',
                   reset='V=Vreset; w+=b',
                   refractory=tarp, method='exponential_euler')

state_mon = StateMonitor(cell, ['V', 'w'], record=True)
spike_mon = SpikeMonitor(cell)

net = Network()
net.add(cell)
net.add(state_mon)
net.add(spike_mon)

In [ ]:
stim_start, stim_dur = 100 * ms, 400 * ms
stim_stop = stim_start + stim_dur

cell.V[0] = EL
I = 0 * pA
net.run(stim_start)
I = params['I_e'] * pA
net.run(stim_dur)
I = 0 * pA
net.run(stim_start)

In [ ]:
spike_times = spike_mon.t / ms
ISI = np.diff(spike_times)
f = 1000 / ISI
print(f'The cell emitted {spike_mon.count[0]} spikes.')
print('\nSpike times:')
print(spike_times)
print('\nInverse of the ISIs:')
print(f)

In [ ]:
time = state_mon.t / ms
V = state_mon.V[0] / mV
w = state_mon.w[0] / pA

In [ ]:
fig,ax = plt.subplots(2, 1, sharex=True)
ax[0].plot(time, V, 'k', lw=1)
ax[1].plot(time, w, 'b', lw=1)
for axx in ax:
    axx.grid(which='major', axis='both', color=[.6,.6,.6], ls=':', lw=0.5)
    for side in 'right','top':
        axx.spines[side].set_visible(False)
ax[0].set_xlim([stim_start/ms - 20, stim_stop/ms + 100])
ax[-1].set_xlabel('Time (ms)')
ax[0].set_ylabel('V (mV)')
ax[1].set_ylabel('w (pA)')
fig.tight_layout()